In [72]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import os
import pickle as pkl
from tqdm import tqdm

In [6]:
path = os.getcwd()
data_dir = path + '/data/'
print(data_dir)

/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/data/


## Read in Reviews

In [13]:
all_reviews = pkl.load(open(data_dir + 'all_reviews.p','rb'))
all_reviews = all_reviews[1:]
print(len(all_reviews))
all_reviews

111092


['Thank you for your help and support!',
 nan,
 'Excellent and professional - highly recommended!',
 'Top illustrator. Superb quality. Fast turn-around. Knows how to translate briefings into visuals. ',
 'What should i say, perfect as always. Thanks :)',
 'Outstanding work, will definitely use again :)',
 'Good provider',
 'excellent work i will work with paul againe',
 'Awesome Awesome Awesome....always fantastic and always will be back',
 'Another great partner',
 'Love working with Blandine! She is a rockstar designer & so professional.',
 'Marcin is the best!',
 'Great design, fantastic job!',
 'The Freelancer helped me with my SEO and set some good backlinks. Will continue to work with him and hopefully get good ranks on Google',
 'we love the design, great work and we will be happy to work with Rali again!!!',
 'Great to work with and made the process and day effortless.',
 'I would recommend JeoFreelance services.',
 'Perfect, thank you. Delivered as promised, A+.',
 'Great desi

## Read in Replacement words

In [21]:
replacement_words_single = pd.read_csv(data_dir+'replacements_singles.csv',index_col=0)
replacement_words_single_no_overlap = pd.read_csv(data_dir+'replacements_minus_flag.csv',index_col=0)

In [25]:
print(replacement_words_single.shape[0])
replacement_words_single.head()

162


,rep_singles
0,kind
1,stable
2,straightforward
3,adapting
4,professional


In [27]:
print(replacement_words_single_no_overlap.shape[0])
replacement_words_single_no_overlap.head()

98


,rep_minus_flag
0,adapting
1,professional
2,compelling
3,observant
4,welcoming


In [36]:
rwsno = list(replacement_words_single_no_overlap.rep_minus_flag)

## Read in Flagged Words

In [34]:
flagged_words_single = pkl.load(open(data_dir + 'flagged_words_single.p','rb'))

## Lemmatize Words Datasets

In [51]:
import nltk
import random
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [54]:
replacement_words_lemmatized = [wordnet_lemmatizer.lemmatize(w) for w in rwsno]
flagged_words_lemmatized = [wordnet_lemmatizer.lemmatize(w) for w in flagged_words_single]

In [57]:
replacement_words_set = set(replacement_words_lemmatized)
flagged_words_set = set(flagged_words_lemmatized)

## Generate Dataframe

In [58]:
columns = ['review','flagged_word','flagged_index','problematic']
df = pd.DataFrame(columns=columns)

In [73]:
flagged_count = 0
replacement_count = 0

for review in tqdm(all_reviews):
    if type(review)==float:
        continue
    review_split = review.split()
    review_lemmatized = [wordnet_lemmatizer.lemmatize(w) for w in review_split]
    
    for i in range(len(review_lemmatized)):
        if review_lemmatized[i] in flagged_words_set:
            #flagged_count+=1
            #print("Flagged:",flagged_count)
            entry = {'review':review,'flagged_word':review_split[i],'flagged_index':i,'problematic':1}
            df = df.append(entry,ignore_index=True)
        elif review_lemmatized[i] in replacement_words_set:
            #replacement_count+=1
            #print("Replacement:",replacement_count)
            entry = {'review':review,'flagged_word':review_split[i],'flagged_index':i,'problematic':0}
            df = df.append(entry,ignore_index=True)

100%|██████████| 111092/111092 [42:19<00:00, 43.75it/s] 


In [74]:
df

,review,flagged_word,flagged_index,problematic
0,Excellent and professional - highly recommended!,professional,2,0
1,Good provider,provider,1,1
2,excellent work i will work with paul againe,excellent,0,1
3,Another great partner,great,1,1
4,"we love the design, great work and we will be happy to work with Rali again!!!",great,4,1
...,...,...,...,...
143317,Shallu was perfectly responsive and did work close with us together to finalize the design. We are looking forward to the next project!,together,10,1
143318,She finish the request with exceptional attention,exceptional,5,1
143319,Great adjustments and feedback along the way. I appreciate the communication throughout.,feedback,3,0
143320,"As usual, Rembrandtjurin (Dennis) did an amazing job! His visual and graphic design skills are the best that I've seen on 99 Designs. He has a super-human eye for design, symmetry and style. This is the second time I've had Dennis work on a project a...",skills,13,1


In [76]:
df_problematic = df[df.problematic==1]
len(df_problematic)

109639

In [77]:
df_neutral = df[df.problematic==0]
len(df_neutral)

33683